In [14]:
from nltk.corpus import wordnet as wn
print wn.synsets('dog')
dog = wn.synsets('dog')[0]
print str(dog.pos())+str(dog.offset())
off = dog.offset()
print wn.synset(str(off))

[Synset('dog.n.01'), Synset('frump.n.01'), Synset('dog.n.03'), Synset('cad.n.01'), Synset('frank.n.02'), Synset('pawl.n.01'), Synset('andiron.n.01'), Synset('chase.v.01')]
n2084071


ValueError: need more than 1 value to unpack

In [2]:
import sys
reload(sys)
sys.setdefaultencoding('utf-8')

读取machine_train_label文件

In [3]:
import codecs
# machine_train_label = codecs.open("./data/labels.csv", "r", "utf-8")
machine_train_label = open("./data/labels.csv", "r")
label_dict = {}
for line in machine_train_label:
    ImageID,Source,LabelName,Confidence = line.split(',')
    if label_dict.has_key(LabelName):
        label_dict[LabelName] += 1
    else:
        label_dict[LabelName] = 1;
machine_train_label.close()

读取machine_valid_label

In [4]:
machine_valid_label = codecs.open("./data/labels_valid.csv", "r", "utf-8")
for line in machine_valid_label:
    ImageID,Source,LabelName,Confidence = line.split(',')
    if label_dict.has_key(LabelName):
        label_dict[LabelName] += 1
    else:
        label_dict[LabelName] = 1;
machine_valid_label.close()

读取humman_valid_label文件

In [5]:
humman_valid_label = codecs.open("./data/labels_humman_valid.csv", "r", "utf-8")
for line in humman_valid_label:
    ImageID,Source,LabelName,Confidence = line.split(',')
    if label_dict.has_key(LabelName):
        label_dict[LabelName] += 1
    else:
        label_dict[LabelName] = 1;
humman_valid_label.close()

输出频数前2000的LabelName

In [7]:
top3000_label = sorted(label_dict.iteritems(), key = lambda d:d[1], reverse=True)
for label in top3000_label:
    print (label[0] + '\t' + str(label[1]) + '\n')

显示前2000标签的分布情况

In [1]:
top3000_label = top3000_label[:2000]
import matplotlib.pyplot as plt
fig = plt.figure(1)
print 'num = '+ str(len(top3000_label))
x = [num for num in range(len(top3000_label))]
y = [label[1] for label in top3000_label]
plt.xlabel('LabelName')
plt.ylabel('Frequency')
plt.plot(x, y)
#plt.xlim(0, 2000)
plt.show()

NameError: name 'top3000_label' is not defined

In [9]:

label_file = codecs.open(u'./data/labels_file.tsv', "a", "utf-8")
num = 0
for label in top3000_label:
    label_file.write(label[0] + '\t' + str(label[1]) + '\n')
    num += 1
label_file.close()

读取dict文件中所有的label名字

In [10]:
dict_file = codecs.open(u'./data/dict.csv', "r")
label_dict = {}
n=0
for line in dict_file:
    print line
    try:
        label_id, label_name = line.split(",", 1)
        label_id = label_id.strip('\t\r\n\"')
        label_name = label_name.strip('\t\r\n\"')
#         print label_id, label_name
        label_dict[label_id] = label_name
    except:
        print n
        print line + ' error'
    n+=1
dict_file.close()

In [11]:
label_match_file = codecs.open(u'./data/labels_match_file.tsv', "a", "utf-8")
print len(top3000_label)
for label in top3000_label:
    try:
#         print (label[0] + '\t' + str(label[1]) + '\t' + label_dict[label[0]] + '\n')
        label_match_file.write(label[0] + '\t' + str(label[1]) + '\t' + label_dict[label[0]] + '\n')
    except:
        print (label[0] + '\t' + str(label[1]) + '\t' + label_dict[label[0]]) 
label_match_file.close()

读取WordNet中的所有Term

wordNet_dict = codecs.open("./data/index.tab", "r", "utf-8")
word_dict = {}
for word in wordNet_dict:
    word = word.split('\t')
    word[0] = word[0].strip('\t\n\r')
    word[1] = word[1].strip('\t\n\r')
    word_dict[word[0]] = word[1]
wordNet_dict.close()

读取WordNet中的所有Sense

1.匹配openimage的前2000的label和wordnet中的word,通过wordnet3.0中的standoff的term数据进行判断是否在wordnet中。

wordnet_openimage_label = codecs.open("./data/wordnet_openimage_label.tsv", "a", "utf-8")
num_in = 0
num_out = 0
num_error = 0
for label in top3000_label:
    label_name_origin = label_dict[label[0]]
    label_name = '_'.join(label_name_origin.split(' '))
    try:
        if label_name in word_dict:
            wordnet_openimage_label.write(label[0] + '\t' + str(label[1]) + '\t' + label_name + '\t'+ word_dict[label_name] + '\n')
            num_in += 1
            #print label_name + '\t' + word_dict[label_name]
        else:
            #去掉复数形式之后在WordNet中
            if label_name[:-1] in word_dict:
                label_name = label_name[:-1]
                wordnet_openimage_label.write(label[0] + '\t' + str(label[1]) + '\t' + label_name + '\t'+ word_dict[label_name] + '\n')
                num_in += 1
                continue
            
            #空格隔开的某一个词在WordNet中
            flag = False
            for word in label_name_origin.split(' '):
                if word in word_dict:
                    label_name = word
                    wordnet_openimage_label.write(label[0] + '\t' + str(label[1]) + '\t' + label_name + '\t'+ word_dict[label_name] + '\n')
                    num_in += 1
                    flag = True
                    break
            if flag:
                continue
            
            #不在WordNet内
            num_out += 1
            wordnet_openimage_label.write(label[0] + '\t' + str(label[1]) + '\t' + label_name + '\t' + '' + '\n')
            #print label_name + '\t' + 'No'
    except:
        print label_name + ' error'
        num_error += 1
wordnet_openimage_label.close()

2.匹配openimage的前2000的label和wordnet中的word,使用nltk中的wordnet工具包来判断label是否在wordnet中

In [12]:
from nltk.corpus import wordnet

In [27]:
%matplotlib inline

In [13]:
print wordnet.synsets('toy_dog')
print 
dog = wordnet.synset('dog.n.01')
print dog.unicode_repr()
print dog.name()
print dog.offset()
print dog.pos()
print dog.lemmas()
print dog.lemma_names()
print dog.lexname()
print dog.definition()
print dog.examples()
print wordnet.ss2of(dog)
print wordnet.of2ss(wordnet.ss2of(dog))

print
dog = wordnet.synset('dog.n.03')
print dog.unicode_repr()
print dog.name()
print dog.offset()
print dog.pos()
print dog.lemmas()
print dog.lemma_names()
print dog.lexname()
print dog.definition()
print dog.examples()

print 
hyp = lambda s:s.hypernyms()
print list(dog.closure(hyp))

print
from pprint import pprint
pprint(dog.tree(hyp))

print wordnet.morphy('dogs')
of = (dog.offset())
print '%09d' % of

In [14]:
wordnet_openimage_description = codecs.open("./data/wordnet_openimage_description.tsv", "a", "utf-8")
des_dict = {}

def getlabelSyn(syns):
    label_syn = []
    for syn in syns:
        wnid = syn.pos() + '%08d' % syn.offset()
        if wnid not in des_dict:
            lex = syn.lexname()
            defin = syn.definition()
            des_dict[wnid] = lex + '\t' + defin
#         label_syn.append(wnid+'||'+lex+'||'+defin)
        label_syn.append(wnid)
    return label_syn

wordnet_openimage_label = codecs.open("./data/wordnet_openimage_label.tsv", "a", "utf-8")
num_in = 0
num_out = 0
num_error = 0
for label in top3000_label:
    label_name_origin = label_dict[label[0]]
    label_name = '_'.join(label_name_origin.split(' ')) 
    try:
        syns = wordnet.synsets(label_name)
        if len(syns):
            label_syn = getlabelSyn(syns)
            wordnet_openimage_label.write(label[0] + '\t' + str(label[1]) + '\t' + label_name_origin + '\t'+ ','.join(label_syn) + '\n')
#             print label[0] + '\t' + str(label[1]) + '\t' + label_name + '\t'+ '\t'.join(label_syn)
            num_in += 1

        else:       
            #空格隔开的某一个词在WordNet中
            flag = False
            label_synsets = []
            for word in label_name_origin.split(' '):
                if len(wordnet.synsets(word)):
                    syns = wordnet.synsets(word)
                    label_syn = getlabelSyn(syns)
                    label_synsets.extend(label_syn)
                    flag = True
            if flag:
                num_in += 1
#                 print label[0] + '\t' + str(label[1]) + '\t' + label_name + '\t'+ '\t'.join(label_synsets)
                wordnet_openimage_label.write(label[0] + '\t' + str(label[1]) + '\t' + label_name_origin + '\t'+ ','.join(label_synsets) + '\n')
                continue
            
            #不在WordNet内
            num_out += 1
#             print label[0] + '\t' + str(label[1]) + '\t' + label_name + '\t'+ ''
            wordnet_openimage_label.write(label[0] + '\t' + str(label[1]) + '\t' + label_name_origin + '\t' + 'not exist' + '\n')
    except:
        print str(label[0]) + ' error'
        print str(label[1])
        print label_name_origin
        wordnet_openimage_label.write(label[0] + '\t' + str(label[1]) + '\t' + label_name_origin + '\t' + ''+ '\n')
        num_error += 1
wordnet_openimage_label.close()

for wnid,des in des_dict.iteritems():
    wordnet_openimage_description.write(wnid + '\t' + des + '\n')
wordnet_openimage_description.close()

In [17]:
print 'haha'
print str(num_in) + ' in WordNet' + '\t' + str(num_out) + ' not in WordNet' + '\t' + str(num_error) + ' error'

不存在wordnet中的label，通过照片去找对应的synset

In [25]:
wordnet_openimage_label = codecs.open("./data/wordnet_openimage_label.tsv", "r", "utf-8")
for line in wordnet_openimage_label:
    lbid, count, label, wnid = line.split('\t')
    wnid = wnid.strip('\n')
    if wnid == "not exist":
        print label
wordnet_openimage_label.close()

提取imageID与labelName的对应字典

In [130]:
imageID_dict = {}

读取machine_train_label文件,提取imageID

In [131]:
machine_train_label = open("./data/labels.csv", "r")
for line in machine_train_label:
    ImageID,Source,LabelName,Confidence = line.split(',')
    if LabelName in imageID_dict:
        imageID_dict[LabelName].add(ImageID)
    else:
        imageID_dict[LabelName] = set(ImageID)
machine_train_label.close()

读取machine_valid_label,提取imageID

In [132]:
machine_valid_label = open("./data/labels_valid.csv", "r")
for line in machine_valid_label:
    ImageID,Source,LabelName,Confidence = line.split(',')
    if LabelName in imageID_dict:
        imageID_dict[LabelName].add(ImageID)
    else:
        imageID_dict[LabelName] = set(ImageID)
machine_valid_label.close()

读取humman_valid_label文件，提取imageID

In [133]:
humman_valid_label = open("./data/labels_humman_valid.csv", "r")
for line in humman_valid_label:
    ImageID,Source,LabelName,Confidence = line.split(',')
    if LabelName in imageID_dict:
        imageID_dict[LabelName].add(ImageID)
    else:
        imageID_dict[LabelName] = set(ImageID)
humman_valid_label.close()

提取前2000label的imageID集合，并去重

In [134]:
top3000_label_imageID = set()
for label in top3000_label:
    for imageID in imageID_dict[label[0]]:
        top3000_label_imageID.add(imageID)
#     top3000_label_imageID = top3000_label_imageID | imageID_dict[label[0]]
print 'top3000_label_imageID size is ' + str(len(top3000_label_imageID))

top3000_label_imageID size is 9016796


In [157]:
print wordnet.synsets('publish')

[Synset('print.v.01'), Synset('publish.v.02'), Synset('publish.v.03')]


In [171]:
print wordnet.lemma('car.n.01.automobile').name
print wordnet.synset('car.n.01').lemma_names

<bound method Lemma.name of Lemma('car.n.01.automobile')>
<bound method Synset.lemma_names of Synset('car.n.01')>
